In [3]:
# 7.3.6 - Pydantic 출력 파서 (Output Parser)
# 구조화된 출력을 위한 Pydantic 모델과 검증 기능 활용

# 필요한 라이브러리 설치
# !pip install langchain==0.2.17 langchain-openai pydantic

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    if api_key:
        os.environ['OPENAI_API_KEY'] = api_key
        print("API 키가 설정되었습니다!")
else:
    print("기존 환경 변수의 API 키를 사용합니다.")

print("=" * 70)
print("7.3.6 Pydantic 출력 파서")
print("=" * 70)

print("""
Pydantic v2 주요 변경사항:
1. @validator → @field_validator로 데코레이터 변경
2. 함수 매개변수명 변경 (v → 실제 필드명)
3. 더 엄격한 타입 검증
4. 성능 향상

핵심 개념:
- 구조화된 출력 보장
- 자동 데이터 검증
- 타입 안전성 제공
- 에러 처리 및 복구
""")

# =============================================================================
# 라이브러리 import
# =============================================================================
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
import json

# LLM 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# =============================================================================
# 기본 Pydantic 모델 정의 (Pydantic v2 기준)
# =============================================================================
print("\n" + "=" * 60)
print("기본 Pydantic 모델: 단어 제안")
print("=" * 60)

class Suggestions(BaseModel):
    """단어 제안을 위한 Pydantic 모델"""
    words: List[str] = Field(description="맥락에 따른 대체 단어들의 리스트")
    
    @field_validator("words")
    @classmethod
    def not_start_with_number(cls, words):
        """단어가 숫자로 시작하지 않는지 검증"""
        for item in words:
            if item and item[0].isnumeric():
                raise ValueError("단어는 숫자로 시작할 수 없습니다!")
        return words

# 파서 생성
parser = PydanticOutputParser(pydantic_object=Suggestions)

# 프롬프트 템플릿 (파서의 형식 지시사항 포함)
prompt = PromptTemplate(
    template="""주어진 단어에 대한 3개의 동의어나 유사한 단어를 제안해주세요.

단어: {word}

{format_instructions}""",
    input_variables=["word"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

print("생성된 형식 지시사항:")
print(parser.get_format_instructions())

# 체인 구성
chain = prompt | llm | parser

# 테스트 실행
test_words = ["artificial", "intelligent", "creative"]

print(f"\n테스트 단어들: {test_words}")

try:
    for word in test_words:
        result = chain.invoke({"word": word})
        print(f"\n단어: {word}")
        print(f"제안된 단어들: {result.words}")
        print(f"결과 타입: {type(result)}")
        
except Exception as e:
    print(f"기본 파서 오류: {e}")

# =============================================================================
# 더 복잡한 Pydantic 모델 예제
# =============================================================================
print("\n" + "=" * 60)
print("복합 Pydantic 모델: 단어 분석")
print("=" * 60)

class WordAnalysis(BaseModel):
    """단어에 대한 종합적 분석 결과"""
    word: str = Field(description="분석 대상 단어")
    definition: str = Field(description="단어의 정의")
    part_of_speech: str = Field(description="품사 (명사, 동사, 형용사 등)")
    synonyms: List[str] = Field(description="동의어 리스트 (최대 3개)", max_length=3)
    example_sentence: str = Field(description="사용 예시 문장")
    difficulty_level: int = Field(description="난이도 (1-10)", ge=1, le=10)
    
    @field_validator("synonyms")
    @classmethod
    def validate_synonyms(cls, synonyms):
        """동의어 검증"""
        if len(synonyms) > 3:
            raise ValueError("동의어는 최대 3개까지만 허용됩니다.")
        return synonyms
    
    @field_validator("part_of_speech")
    @classmethod
    def validate_pos(cls, pos):
        """품사 검증"""
        allowed_pos = ["명사", "동사", "형용사", "부사", "전치사", "접속사", "감탄사"]
        if pos not in allowed_pos:
            raise ValueError(f"허용된 품사가 아닙니다: {allowed_pos}")
        return pos

# 복합 분석 파서
analysis_parser = PydanticOutputParser(pydantic_object=WordAnalysis)

# 복합 분석 프롬프트
analysis_prompt = PromptTemplate(
    template="""다음 단어에 대한 종합적인 분석을 제공해주세요:

단어: {word}

다음 정보를 모두 포함해주세요:
- 한국어 정의
- 품사 (명사, 동사, 형용사, 부사, 전치사, 접속사, 감탄사 중 하나)
- 동의어 3개 이하
- 사용 예시 문장
- 난이도 (1-10)

{format_instructions}""",
    input_variables=["word"],
    partial_variables={"format_instructions": analysis_parser.get_format_instructions()}
)

# 복합 분석 체인
analysis_chain = analysis_prompt | llm | analysis_parser

print("복합 분석 테스트: 'technology'")

try:
    analysis_result = analysis_chain.invoke({"word": "technology"})
    
    print(f"\n=== 단어 분석 결과 ===")
    print(f"단어: {analysis_result.word}")
    print(f"정의: {analysis_result.definition}")
    print(f"품사: {analysis_result.part_of_speech}")
    print(f"동의어: {', '.join(analysis_result.synonyms)}")
    print(f"예시 문장: {analysis_result.example_sentence}")
    print(f"난이도: {analysis_result.difficulty_level}/10")
    
except Exception as e:
    print(f"복합 분석 오류: {e}")

# =============================================================================
# 선택적 필드가 있는 모델
# =============================================================================
print("\n" + "=" * 60)
print("선택적 필드 모델: 제품 리뷰 분석")
print("=" * 60)

class ProductReview(BaseModel):
    """제품 리뷰 분석 결과"""
    product_name: str = Field(description="제품명")
    rating: int = Field(description="평점 (1-5)", ge=1, le=5)
    sentiment: str = Field(description="감정 (긍정, 부정, 중립)")
    key_points: List[str] = Field(description="주요 포인트들")
    recommendation: Optional[str] = Field(None, description="추천 여부 (선택사항)")
    
    @field_validator("sentiment")
    @classmethod
    def validate_sentiment(cls, sentiment):
        allowed_sentiments = ["긍정", "부정", "중립"]
        if sentiment not in allowed_sentiments:
            raise ValueError(f"허용된 감정: {allowed_sentiments}")
        return sentiment

# 제품 리뷰 파서
review_parser = PydanticOutputParser(pydantic_object=ProductReview)

# 리뷰 분석 프롬프트
review_prompt = PromptTemplate(
    template="""다음 제품 리뷰를 분석해주세요:

리뷰 텍스트: "{review_text}"

다음 정보를 추출해주세요:
- 제품명
- 평점 (1-5)
- 감정 (긍정, 부정, 중립)
- 주요 포인트들
- 추천 여부 (선택사항)

{format_instructions}""",
    input_variables=["review_text"],
    partial_variables={"format_instructions": review_parser.get_format_instructions()}
)

# 리뷰 분석 체인
review_chain = review_prompt | llm | review_parser

# 샘플 리뷰
sample_review = """
아이폰 15는 정말 훌륭한 스마트폰입니다. 
카메라 품질이 이전 모델보다 훨씬 좋아졌고, 
배터리 수명도 만족스럽습니다. 
다만 가격이 좀 비싸긴 하지만 그만한 값어치는 하는 것 같아요.
전반적으로 추천합니다!
"""

print("리뷰 분석 테스트:")
print(f"샘플 리뷰: {sample_review.strip()}")

try:
    review_result = review_chain.invoke({"review_text": sample_review})
    
    print(f"\n=== 리뷰 분석 결과 ===")
    print(f"제품명: {review_result.product_name}")
    print(f"평점: {review_result.rating}/5")
    print(f"감정: {review_result.sentiment}")
    print(f"주요 포인트:")
    for i, point in enumerate(review_result.key_points, 1):
        print(f"  {i}. {point}")
    if review_result.recommendation:
        print(f"추천: {review_result.recommendation}")
    
except Exception as e:
    print(f"리뷰 분석 오류: {e}")

# =============================================================================
# 에러 처리 및 재시도 로직
# =============================================================================
print("\n" + "=" * 60)
print("에러 처리 및 재시도")
print("=" * 60)

def safe_parse_with_retry(chain, input_data, max_retries=3):
    """파싱 실패 시 재시도하는 안전한 파서"""
    for attempt in range(max_retries):
        try:
            result = chain.invoke(input_data)
            print(f"✅ 시도 {attempt + 1}회 성공")
            return result
        except Exception as e:
            print(f"⚠️ 시도 {attempt + 1}회 실패: {e}")
            if attempt == max_retries - 1:
                print("❌ 모든 재시도 실패")
                return None
    return None

# 에러 처리 테스트
print("에러 처리 테스트:")
test_result = safe_parse_with_retry(
    chain, 
    {"word": "computer"}, 
    max_retries=2
)

if test_result:
    print(f"최종 결과: {test_result.words}")

# =============================================================================
# 활용 팁 및 베스트 프랙티스
# =============================================================================
print("\n" + "=" * 60)
print("Pydantic 출력 파서 활용 팁")
print("=" * 60)

print("""
Pydantic v2 주요 개선사항:
1. @field_validator: 더 명확한 검증 로직
2. 타입 힌트 강화: 더 정확한 타입 체킹
3. 성능 향상: 더 빠른 검증 속도
4. 에러 메시지 개선: 더 명확한 오류 정보

베스트 프랙티스:
1. 필드 설명 상세히 작성 (description)
2. 적절한 제약조건 설정 (ge, le, max_length 등)
3. 커스텀 검증 로직 활용 (@field_validator)
4. 선택적 필드 적절히 활용 (Optional)
5. 에러 처리 및 재시도 로직 구현

실제 활용 사례:
- API 응답 구조화
- 데이터 추출 및 정제
- 자동 검증 및 품질 관리
- 구조화된 보고서 생성
- 다국어 번역 결과 포맷팅
""")

print("\n🎉 7.3.6 Pydantic 출력 파서 예제 완료!")

기존 환경 변수의 API 키를 사용합니다.
7.3.6 Pydantic 출력 파서

Pydantic v2 주요 변경사항:
1. @validator → @field_validator로 데코레이터 변경
2. 함수 매개변수명 변경 (v → 실제 필드명)
3. 더 엄격한 타입 검증
4. 성능 향상

핵심 개념:
- 구조화된 출력 보장
- 자동 데이터 검증
- 타입 안전성 제공
- 에러 처리 및 복구


기본 Pydantic 모델: 단어 제안
생성된 형식 지시사항:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "단어 제안을 위한 Pydantic 모델", "properties": {"words": {"description": "맥락에 따른 대체 단어들의 리스트", "items": {"type": "string"}, "title": "Words", "type": "array"}}, "required": ["words"]}
```

테스트 단어들: ['artificial', 'intelligent', 'creative']

단어: artificial
제안된 단어들: ['synthetic', 'man-